In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
!pip install wtfml
!pip install efficientnet_pytorch

In [ ]:
import warnings
import torch_xla
import torch_xla.debug.metrics as met
import torch_xla.distributed.data_parallel as dp
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm 
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.test.test_utils as test_utils
import warnings
import gc

warnings.filterwarnings("ignore")

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision

import cv2

import numpy as np 
import pandas as pd
import os

from torch.utils.data import DataLoader,TensorDataset,Dataset
import matplotlib.pyplot as plt
import albumentations
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
from efficientnet_pytorch import EfficientNet

from wtfml.utils import EarlyStopping

In [ ]:
data = pd.read_csv('/kaggle/input/melanoma-merged-external-data-512x512-jpeg/folds_13062020.csv')
print(data.columns)

benign_data = data[(data['source']== 'ISIC20') & (data['target']==0)]
malign_data = data[data['target']==1]
data = benign_data.append(malign_data)

data.drop('source',axis=1,inplace=True)
data.to_csv('train_data.csv',index=False)

del data, benign_data, malign_data

In [ ]:
import sys
sys.path.append('../input/autoaug')
from auto_augment import AutoAugment, Cutout


In [ ]:
train_path = '/kaggle/input/melanoma-merged-external-data-512x512-jpeg/512x512-dataset-melanoma/512x512-dataset-melanoma/'
# test_path = '/kaggle/input/melanoma-merged-external-data-512x512-jpeg/512x512-test/512x512-test/'
df= pd.read_csv('./train_data.csv')

In [ ]:
df.head()

In [ ]:
df['fold'] = -1
Y = df['target'].values
kf = model_selection.StratifiedKFold(n_splits=3,shuffle=True)
idx = kf.get_n_splits(X=df,y=Y)
print(idx)
for fold,(x,y) in enumerate(kf.split(X=df,y=Y)):
    df.loc[y,'fold'] = fold
df.head()

In [ ]:
df.columns =['image_name', 'patient_id', 'target', 'sex', 'age_approx',
       'anatom_site_general_challenge', 'stratify_group', 'kfold']
df.head()

In [ ]:
class CustomDataset(Dataset):
    def __init__(self,path,name,target,aug):
        super(CustomDataset,self).__init__()
        self.path = path
        self.name = name
        self.target = target
        self.aug = aug
        
        
    def __len__(self):
        return len(self.name)
    
    def __getitem__(self,index):
        
        im_name = self.name[index]
        y = self.target[index]
        img_path = os.path.join(self.path,im_name + '.jpg')
        img = cv2.resize(cv2.imread(img_path),dsize=(384,384))
        image = self.aug(image=img)
        l = image['image']
        image = np.transpose(l, (2, 0, 1)).astype(np.float32)
        
        return torch.tensor(image,dtype=torch.float),torch.tensor(y)

In [ ]:
class Data_Loader():
    def __init__(self,path,name,target,aug):
        self.path = path
        self.name = name
        self.target = target
        self.aug = aug
        self.dataset = CustomDataset(self.path,self.name,self.target,self.aug)
        
    def get(self,batch_size,shuffle,num_workers):
        
        sampler = torch.utils.data.distributed.DistributedSampler(self.dataset,
                                                                  num_replicas = xm.xrt_world_size(),
                                                                  rank = xm.get_ordinal(),
                                                                  shuffle = shuffle)
        dataloader = torch.utils.data.DataLoader(self.dataset,
                                                 batch_size=batch_size,
                                                 shuffle=False,
                                                 sampler=sampler,
                                                 num_workers=num_workers)
        return dataloader
       

In [ ]:
class EffNet(nn.Module):
    def __init__(self,model='b0'):
        super(EffNet,self).__init__()
        
        model_name = 'efficientnet' + model
        self.feature = EfficientNet.from_pretrained("efficientnet-b1")
        self.drop = nn.Dropout(0.25)
        self.l0 = nn.Linear(1280,1) #b4-1792 # b3 - 1536, b2 - 1408, b0 = 1280, b1 = 1280
        self.l1 = nn.Sigmoid()
        
        
        
    def forward(self,img):
        batch_size = img.shape[0]
        
        x = self.feature.extract_features(img)
        #print(x.shape)
        
        x = nn.functional.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
        #print(x.shape)
        
        x = self.drop(x)
        #print(x.shape)
        out = self.l0(x)
        #print(out.shape)
        out = self.l1(out)
        
        return out

In [ ]:
def train(model,fold):
    
    batch_t = 128
    batch_v = 128
    best_score = 0
    device = xm.xla_device() 
    
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    train_transpose = albumentations.Compose([
                albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),
                albumentations.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15),
                albumentations.Flip(p=0.5)
            ])
    valid_transpose = albumentations.Compose(
        [ albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True) 
        ])
    
    image_path = '../input/melanoma-merged-external-data-512x512-jpeg/512x512-dataset-melanoma/512x512-dataset-melanoma/' #'/kaggle/input/siic-isic-224x224-images/train/'
    train_df = df[df.kfold!=fold].reset_index(drop=True)  #kfold to fold
    valid_df = df[df.kfold==fold].reset_index(drop=True)
    train_im = train_df.image_name.values.tolist()
    train_y = train_df.target.values
    valid_im = valid_df.image_name.values.tolist()
    valid_y = valid_df.target.values
    train_dataset = Data_Loader(image_path,train_im,train_y,
                                train_transpose).get(batch_size=batch_t,shuffle=True,num_workers=4)
    valid_dataset = Data_Loader(image_path,valid_im,valid_y,
                               valid_transpose).get(batch_size=batch_v,shuffle=False,num_workers=4)
    
    
    
    
    
    #model = Resnext50_32x4d()
    #model = EffNet()
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    schedular = torch.optim.lr_scheduler.ReduceLROnPlateau(    #to update the learning rate if model auc score does not increase
        optimizer,                                             #for 3 succesive epochs
        patience=3,           
        threshold=0.001,
        mode="max"
    )

    es = EarlyStopping(patience=5, mode="max",tpu=True)  #early stopping function to stop training if auc score does not increase over 5 epochs
    criterion = nn.BCEWithLogitsLoss()
    #criterion = FocalLoss()
    epochs = 10
    best_score = 0
    
    
    
    for epoch in range(epochs):
            #train mode for training the model and updating the losses
            model.train()
            batch = 0
            #para_loader = pl.ParallelLoader(train_dataset,[device])
            #train_loader = para_loader.per_device_loader(device)
        
            #for _,(train_data,label) in enumerate(train_loader):
            for train_data,label in train_dataset:
                train_data = train_data.to(device)
                label = torch.tensor(label,dtype = torch.float32)
                label = label.to(device)
                
                optimizer.zero_grad()
                out = model(train_data)
                loss = criterion(out,label.unsqueeze(1).type_as(out))
                #loss = criterion(out,label)
                batch +=1
                del train_data,label
                gc.collect()
                if batch%100==0 : print("EPOCH {}  Loss {}  batch  {}".format(epoch,loss.item(),batch))
                
                loss.backward()
                xm.optimizer_step(optimizer,barrier=True)
            #evaluate mode to evaluate the model on cv and update learning rate based on auc score
            #del para_loader,train_loader
            gc.collect()
            model.eval()
            preds = []
            batch = 0
            #para_loader = pl.ParallelLoader(valid_dataset,[device])
            #valid_loader = para_loader.per_device_loader(device)
            
            #for _,(valid_data,valid_label) in enumerate(valid_dataset):
            for valid_data,valid_label in valid_dataset:
                valid_data = valid_data.to(device)
                valid_label = torch.tensor(valid_label,dtype = torch.float32)
                valid_label = valid_label.to(device)
                batch +=1
                
                
                with torch.no_grad():
                    out = model(valid_data)
                    #loss = criterion(out,valid_label)
                    loss = criterion(out,valid_label.unsqueeze(1).type_as(out))
                    preds.append(out.cpu())
                    if batch%50==0 : xm.master_print('Valid Loss {}  batch  {}'.format(loss.item(),batch))
                del valid_data,valid_label
                gc.collect()
            #del para_loader,valid_loader
            gc.collect()
            pred=np.vstack((preds)).ravel()
            #print('pred',pred)
            auc_score = roc_auc_score(valid_y.astype(np.float32),pred)
            print("EPOCH {}  AUC Score {}".format(epoch,auc_score))
            schedular.step(auc_score)
            es(auc_score, model, model_path=f"model_fold_{fold}.bin")
            if es.early_stop:
                print("Early stopping")
                break
            gc.collect()

In [ ]:
model = EffNet()

In [ ]:
train(model,0)
train(model,1)
train(model,2)

## Prediction

In [ ]:
def predict(fold):
    test_df = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/test.csv')   #/kaggle/input/siim-isic-melanoma-classification/test.csv
    im_path = '/kaggle/input/melanoma-merged-external-data-512x512-jpeg/512x512-test/512x512-test/' #'/kaggle/input/siic-isic-224x224-images/test/'
    batch_t = 32
    #model_path = '../working/model_fold_'+str(fold)+'.bin'
    model_path = '../working/model_fold_'+str(fold)+'.bin' 
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    test_transpose = albumentations.Compose(
        [ albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True) 
        ])
    
    test_im = test_df.image_name.values.tolist()
    test_y = np.ones(len(test_im))
    test_dataset = Data_Loader(im_path,test_im,test_y,test_transpose).get(batch_t,shuffle=False,num_workers=4)
    device = xm.xla_device()
    
    
    model = EffNet()
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    preds = []
    batch = 0
    #for i in range(5):
    for test_data,test_label in test_dataset:
                test_data = test_data.to(device)
                batch +=1
                
                with torch.no_grad():
                    out = model(test_data)
                    preds.append(out.cpu())
                    if batch%50==0 : print('Batch  {}'.format(batch))

    pred=np.vstack((preds)).ravel()
    return pred

In [ ]:
predict_1 = predict(0)
predict_2 = predict(1)
predict_3 = predict(2)

In [ ]:
sub = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')
sub['target'] = predict_1
sub.to_csv('b1_signmoid_1_fold.csv',index=False)
# sub = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')
sub['target'] = predict_2
sub.to_csv('b1_signmoid_2_fold.csv',index=False)

sub['target'] = predict_3
sub.to_csv('b1_signmoid_3_fold.csv',index=False)

sub['target'] = (predict_1 + predict_2 + predict_3)/3
sub.to_csv('b1_signmoid_3folds_avg.csv',index=False)